# Tuning Neural Networks with Normalization - Lab

## Introduction

For this lab on initialization and optimization, you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives
You will be able to:
* Build a neural network using Keras
* Normalize your data to assist algorithm convergence
* Implement and observe the impact of various initialization techniques

In [67]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

## Loading the data

The data we'll be working with is data related to Facebook posts published during the year of 2014 on the Facebook page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set, `dataset_Facebook.csv`, and delete any rows with missing data. Afterwards, briefly preview the data.

In [68]:
#Your code here; load the dataset and drop rows with missing values. Then preview the data.
df = pd.read_csv('dataset_Facebook.csv', sep=';')
df.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [69]:
df.isna().sum()

Page total likes                                                       0
Type                                                                   0
Category                                                               0
Post Month                                                             0
Post Weekday                                                           0
Post Hour                                                              0
Paid                                                                   1
Lifetime Post Total Reach                                              0
Lifetime Post Total Impressions                                        0
Lifetime Engaged Users                                                 0
Lifetime Post Consumers                                                0
Lifetime Post Consumptions                                             0
Lifetime Post Impressions by people who have liked your Page           0
Lifetime Post reach by people who like your Page   

In [70]:
df.dropna(inplace=True)

In [71]:
df.isna().sum()

Page total likes                                                       0
Type                                                                   0
Category                                                               0
Post Month                                                             0
Post Weekday                                                           0
Post Hour                                                              0
Paid                                                                   0
Lifetime Post Total Reach                                              0
Lifetime Post Total Impressions                                        0
Lifetime Engaged Users                                                 0
Lifetime Post Consumers                                                0
Lifetime Post Consumptions                                             0
Lifetime Post Impressions by people who have liked your Page           0
Lifetime Post reach by people who like your Page   

## Defining the Problem

Define X and Y and perform a train-validation-test split.

X will be:
* Page total likes
* Post Month
* Post Weekday
* Post Hour
* Paid
along with dummy variables for:
* Type
* Category

Y will be the `like` column.

In [72]:
#Your code here; define the problem.
X = df[['Page total likes', 'Post Month', 'Post Weekday', 'Post Hour','Paid','Type','Category']]

y = df[['like']]


In [73]:
# Define dummy variables

Type_dummies = pd.get_dummies(X['Type'], prefix='Type_', drop_first=True)
Category_dummies = pd.get_dummies(X['Category'], prefix='Cat_', drop_first=True)

In [74]:
X = pd.concat([X, Type_dummies, Category_dummies], axis=1)

In [75]:
X.drop(['Type','Category'], inplace=True, axis=1)

In [76]:
X_train, X_val, Y_train, Y_val = train_test_split(X,y, random_state=123)

In [77]:
len(X_train.columns)

10

## Building a Baseline Model

Next, build a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model.

In [78]:
#Simply run this code block, later you'll modify this model to tune the performance
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

### Evaluating the Baseline

Evaluate the baseline model for the training and validation sets.

In [79]:
#Your code here; evaluate the model with MSE
model.evaluate(X_val, Y_val)

124/124 [==============================] - 0s 14us/step


[nan, nan]

In [80]:
#Your code here; inspect the loss function through the history object
print(model.history.history)

{'val_loss': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'val_mse': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

> Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.

## Normalize the Input Data

Normalize the input features by subtracting each feature mean and dividing by the standard deviation in order to transform each into a standard normal distribution. Then recreate the train-validate-test sets with the transformed input data.

In [86]:
## standardize/categorize

#Your code here; define the problem.
X_to_scale = df[['Page total likes', 'Post Month', 'Post Weekday', 'Post Hour','Paid']]



495

In [82]:
scaler = StandardScaler()

In [83]:
X_scaled = scaler.fit_transform(X_to_scale)
X_scaled = pd.DataFrame(X_scaled, columns=X_to_scale.columns)

In [101]:
X_scaled.reset_index(inplace=True)
Type_dummies.reset_index(inplace=True)
Category_dummies.reset_index(inplace=True)

In [106]:
X_norm = pd.concat([X_scaled,Type_dummies, Category_dummies],axis=1)
X_norm.drop(['index','index','index'],axis=1, inplace=True)

In [107]:
X_train, X_val, Y_train, Y_val = train_test_split(X_norm,y, random_state=123)

In [108]:
X_train.head()

,Page total likes,Post Month,Post Weekday,Post Hour,Paid,Type__Photo,Type__Status,Type__Video,Cat__2,Cat__3
148,0.774658,0.900265,0.427207,0.720342,-0.62486,1,0,0,1,0
454,-1.881228,-1.523291,-1.051585,1.176897,-0.62486,1,0,0,0,1
467,-1.967159,-1.523291,-0.558655,-1.105878,-0.62486,1,0,0,0,1
422,-1.386338,-1.220346,1.413068,2.318285,-0.62486,1,0,0,0,0
31,0.971230,1.506154,-0.558655,0.492065,-0.62486,1,0,0,1,0


## Refit the Model and Reevaluate

Great! Now refit the model and once again assess it's performance on the training and validation sets.

In [109]:
#Your code here; refit a model as shown above
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [110]:
#Rexamine the loss function
model.evaluate(X_val, Y_val)

124/124 [==============================] - 0s 20us/step


[nan, nan]

> Note that you still haven't achieved convergence! From here, it's time to normalize the output data.

## Normalizing the output

Normalize Y as you did X by subtracting the mean and dividing by the standard deviation. Then, resplit the data into training and validation sets as we demonstrated above, and retrain a new model using your normalized X and Y data.

In [112]:
#Your code here: redefine Y after normalizing the data.
y = scaler.fit_transform(y)

In [113]:
#Your code here; create training and validation sets as before. Use random seed 123.
X_train, X_val, Y_train, Y_val = train_test_split(X_norm,y, random_state=123)

In [114]:
#Your code here; rebuild a simple model using a relu layer followed by a linear layer. (See our code snippet above!)
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

Again, reevaluate the updated model.

In [115]:
#Your code here; MSE
model.evaluate(X_val, Y_val)

124/124 [==============================] - 0s 17us/step


[0.8092441265621493, 0.8092441558837891]

In [116]:
#Your code here; loss function
model.history.history

{'val_loss': [1.072243304983262,
  0.9880959910731162,
  0.937640167051746,
  0.9068550491525281,
  0.884835148049939,
  0.8688690571054336,
  0.8569619131665076,
  0.8474722288308605,
  0.8404915808669983,
  0.8351031820620259,
  0.8307577919575476,
  0.827183207196574,
  0.8246236731929164,
  0.8225529717822229,
  0.8201591213864665,
  0.8184252386131594,
  0.8172099085584763,
  0.8157566478175502,
  0.8145006457644124,
  0.8134670882455765,
  0.8128812587068927,
  0.8121800946612512,
  0.8134782847858244,
  0.8125353282497775,
  0.8144711900141931,
  0.8158704249128219,
  0.8145379241435758,
  0.8135495964557894,
  0.8125210082338702,
  0.811767648785345,
  0.8107859920109471,
  0.8102614826733067,
  0.80978300782942,
  0.8096006911608481,
  0.8107367383856927,
  0.8105795657442462,
  0.810079006418105,
  0.8097234252960451,
  0.8094322450699345,
  0.8091095416776596,
  0.8084171750853139,
  0.8085713737433956,
  0.8099787076634746,
  0.809416568567676,
  0.8095310931244204,
  0.809

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.)

## Using Weight Initializers

Below, take a look at the code provided to see how to modify the neural network to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe is the strongest.

##  He Initialization

In [117]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, kernel_initializer= "he_normal",
                activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val),verbose=0)

In [118]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [119]:
print(MSE_train)
print(MSE_val)

1.1509542496659626
0.837949659950908


## Lecun Initialization

In [120]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, 
                kernel_initializer= "lecun_normal", activation='tanh'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [121]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [122]:
print(MSE_train)
print(MSE_val)

1.142327939812121
0.8345677505829496


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop

In [123]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "rmsprop" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [124]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [125]:
print(MSE_train)
print(MSE_val)

1.13785499321751
0.8388479715955445


## Adam

In [126]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "Adam" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [127]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [128]:
print(MSE_train)
print(MSE_val)

1.1443077180720171
0.8427957579410955


## Learning Rate Decay with Momentum


In [129]:
np.random.seed(123)
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= sgd ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [130]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [131]:
print(MSE_train)
print(MSE_val)

1.2517355675120523
0.954668314925454


## Selecting a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [ ]:
#Your code here
# RMS Prop achieves the lowest MSE on the validation set, and a 

## Summary  

In this lab, you worked to ensure your model converged properly. Additionally, you also investigated the impact of varying initialization and optimization routines.